# ONNX export + INT8 quantization + Benchmark

Bu notebook layihənin “sürət və optimizasiya” hissəsini bağlayır.

Məqsəd:
- PyTorch modelləri ONNX formatına çevirmək (CPU inference üçün daha sürətli)
- ONNX modelləri INT8 (dynamic quantization) ilə sıxmaq
- Ölçü və sürət müqayisəsi aparmaq (PyTorch vs ONNX vs ONNX INT8)

Bu hissə çox vacibdir, çünki təkcə "model varmı?" sualına yox, "model CPU-da necə sürətli işləyir?" göstərir.

In [1]:
from pathlib import Path
import os, sys, subprocess

Az əvvəlki Python skriptini yenə run edirik:

In [2]:
ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

os.chdir(ROOT)
print("Repo root:", Path.cwd())


Repo root: d:\github_repos\kontakt_home_task\task3


Script və modul run-ları üçün ən stabil variant `PYTHONPATH`a **src** əlavə etmək

In [3]:
PYTHONPATH = str(Path.cwd() / "src")
ENV = os.environ.copy()
ENV["PYTHONPATH"] = PYTHONPATH + (os.pathsep + ENV["PYTHONPATH"] if ENV.get("PYTHONPATH") else "")

def run(cmd):
    """Always run with the same python as this notebook kernel + correct PYTHONPATH."""
    if isinstance(cmd, str):
        cmd = cmd.split()
    print("RUN:", " ".join(cmd))
    return subprocess.run(cmd, env=ENV, check=True)

### Bu notebook-da niyə `run()` istifadə edirəm?

Eyni səbəb: notebook hansı Python interpreter ilə açılıbsa, bütün komandalar da həmin interpreter ilə işləsin.

Əks halda:
- `python` PATH-dan başqa yerə düşə bilər
- `pii_guard` import problemi çıxa bilər
- ONNX faylları başqa environmentdə yazılıb burada tapılmaya bilər

## 1) Export ONNX

Bu nə üçündür?

PyTorch modeli birbaşa CPU-da işləyə bilər, amma ONNX Runtime çox vaxt daha sürətli olur.

Export prosesində:
- `models/classifier/pytorch`  →  `models/classifier/onnx`
- `models/ner/pytorch`         →  `models/ner/onnx`

Nəticədə `model.onnx` və tokenizer/config faylları həmin qovluqlarda olur.


In [4]:
run([sys.executable, "-m", "pii_guard.optimization.export_onnx"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pii_guard.optimization.export_onnx


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pii_guard.optimization.export_onnx'], returncode=0)

## 2) Quantize INT8

In [5]:
run([sys.executable, "-m", "pii_guard.optimization.quantize"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pii_guard.optimization.quantize


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pii_guard.optimization.quantize'], returncode=0)

### Bəs INT8 Quantization nə üçündür?

INT8 quantization modelin çəkilərini 8-bit formatına salır.

Praktik effekt budur ki,
- model ölçüsü ciddi kiçilir (məs: 500MB → 120-130MB)
- inference CPU-da daha sürətli olur

Burada dynamic quantization istifadə olunur.
- Bu sayədə həm ataset calibration tələb etmir
- həm də pipeline üçün rahatdır

## 3) Benchmark (n=80)

In [6]:
run([sys.executable, "-m", "pii_guard.optimization.benchmark", "--n", "80"])

RUN: d:\github_repos\kontakt_home_task\.venv\Scripts\python.exe -m pii_guard.optimization.benchmark --n 80


CompletedProcess(args=['d:\\github_repos\\kontakt_home_task\\.venv\\Scripts\\python.exe', '-m', 'pii_guard.optimization.benchmark', '--n', '80'], returncode=0)

## 4) Report-u cədvəl kimi göstərək

In [7]:
import json
from pathlib import Path


In [8]:
try:
    import pandas as pd
except Exception:
    run([sys.executable, "-m", "pip", "install", "-q", "pandas"])
    import pandas as pd

bench = json.loads(Path("reports/benchmarks/bench.json").read_text(encoding="utf-8"))

In [9]:
sizes = pd.Series(bench["sizes_mb"], name="size_mb").to_frame().sort_values("size_mb")
lat = pd.Series(bench["latency_ms_avg"], name="latency_ms_avg").to_frame().sort_values("latency_ms_avg")

display(sizes)
display(lat)

,size_mb
ner_onnx_int8,128.870521
classifier_onnx_int8,129.432046
ner_onnx,514.102571
classifier_onnx,516.335464


,latency_ms_avg
classifier_onnx_int8_safe,12.456330
ner_onnx_int8,19.149159
ner_onnx,43.471030
classifier_onnx_safe,50.442089
classifier_pytorch_unsafe,109.753114
ner_pytorch,124.147169
classifier_pytorch_safe,149.382431


Bu hissədə JSON-u oxuyub daha rahat görmək üçün cədvəl kimi göstərirəm.

Məqsəd həm hansı variantın daha sürətli olduğunu, həm də ölçünün necə dəyişdiyini konkret rəqəmlərlə göstərməkdir.

In [10]:
pt = bench["latency_ms_avg"]["classifier_pytorch_safe"]
onnx = bench["latency_ms_avg"]["classifier_onnx_safe"]
int8 = bench["latency_ms_avg"]["classifier_onnx_int8_safe"]
print(f"Guardrail speedup: ONNX={pt/onnx:.2f}x, INT8={pt/int8:.2f}x")

Guardrail speedup: ONNX=2.96x, INT8=11.99x


## Qısa yekun (real sistemdə nə alınır?)

Bu notebook-un nəticəsi adətən (*adətən* dedim, çünki GPU-nun olub-olmaması nəticəni qismən dəyişə bilər) belə olur:

- ONNX, PyTorch-dan xeyli sürətli çıxır
- INT8 ONNX ən sürətli variant olur
- Model ölçüsü də ciddi azalır

Bu da Guardrail üçün 20ms-dən aşağı” latency hədəfinə yaxınlaşmağın ən real yoludur (CPU mühitində :D).
